In [2]:
## importing all the neccessary package for this project
import numpy as np
import pandas as pd
import time
import datetime
from datetime import datetime, timezone
import requests
import json
import tweepy
import io
from io import BytesIO
import os
import re

<style>
    /* define the default link style */
    a {
        color: gray;
        text-decoration: none;
    }
    
    /* define the link style when it is clicked */
    a.clicked {
        color: blue;
    }
    
    /* define the link style when it is hovered */
    a:hover {
        color: black;
    }
</style>

<ul>
    <li><a href="#Gather"><strong><font size="+2">Gather</font></strong></a></li>
    <li><a href="#Assess"><strong><font size="+2">Assess</font></strong></a></li>
    <li><a href="#Clean"><strong><font size="+2">Clean</font></strong></a></li>
    <li><a href="#Store"><strong><font size="+2">Store</font></strong></a></li>
</ul>


<a id='Gather'></a>
# **Gather**
There are three data to collect for this analysis.

first is the "twitter_archive_enhanced" in CSV,

then, the "image_predictions" in TSV in URL,

lastly, the query the Twitter API and collect Data on retweet_count and like_count. (This has been provided by Udacity instructor since twitter has failed to provide me with an API access)

In [3]:
## load the twitter_archive_enhanced.csv file on df using pandas
filepath = "twitter-archive-enhanced-2 (2).csv"
twitter_archive_df = pd.read_csv(filepath)                         
twitter_archive_df.head(2)


tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   

   rating_denominator     name doggo floofer pupper puppo  
0                  10  Phineas  None    None   None  None  
1                  10    Tilly  None    None   None  None

In [4]:
twitter_archive_df.tail(2)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
2354  666029285002620928                    NaN                  NaN   
2355  666020888022790149                    NaN                  NaN   

                      timestamp  \
2354  2015-11-15 23:05:30 +0000   
2355  2015-11-15 22:32:08 +0000   

                                                 source  \
2354  <a href="http://twitter.com/download/iphone" r...   
2355  <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
2354  This is a western brown Mitsubishi terrier. Up...                  NaN   
2355  Here we have a Japanese Irish Setter. Lost eye...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
2354                       NaN                        NaN   
2355                       NaN                        NaN   

                                          expanded_urls  rating_numerator  \
2354  https://twitter.com/dog_rates/status/666029285...                 7   
2355  https://twitter.com/dog_rates/status/666020888...                 8   

      rating_denominator  name doggo floofer pupper puppo  
2354                  10     a  None    None   None  None  
2355                  10  None  None    None   None  None

In [5]:
## download the image_prediction TSV file from the file using request

url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
response = requests.get(url)

# Opening a tsv file and saving the response content
with open('image-predictions.tsv', mode='wb') as file:
    file.write(response.content)
    
# Read TSV file
image_prediction_df = pd.read_csv('image-predictions.tsv', sep='\t')
# Looking at the information in our data
image_prediction_df.head(3)

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True

In [6]:
### create an empty list, read the tweet_json file, use "for loop" to read it line by line.
### create a list of the required data from the json file

### create an empty list
more_data = []

## using os.path.expanduser to locate tweet-json text file
filepath_tweet = os.path.join(os.path.expanduser("~/Desktop/WRD"), "tweet_json.txt")

### open and create a dictionary using "for loop" to read file line by line from the "tweet_json.txt" to extract the tweet id, Retweet_count and Tweet_id  
with open(filepath_tweet, 'r', encoding = 'utf-8') as file:
    for line in file:
        each_tweet = json.loads(line)
        tweet_id = each_tweet['id']
        favorite_count = each_tweet['favorite_count']
        retweet_count = each_tweet['retweet_count']
        ### append tweets, 
        more_data.append({'tweet_id':tweet_id, 'retweet_count':retweet_count, 'favorite_count':favorite_count})

more_data## more_data is in dictionary  format

[{'tweet_id': 892420643555336193,
  'retweet_count': 8853,
  'favorite_count': 39467},
 {'tweet_id': 892177421306343426,
  'retweet_count': 6514,
  'favorite_count': 33819},
 {'tweet_id': 891815181378084864,
  'retweet_count': 4328,
  'favorite_count': 25461},
 {'tweet_id': 891689557279858688,
  'retweet_count': 8964,
  'favorite_count': 42908},
 {'tweet_id': 891327558926688256,
  'retweet_count': 9774,
  'favorite_count': 41048},
 {'tweet_id': 891087950875897856,
  'retweet_count': 3261,
  'favorite_count': 20562},
 {'tweet_id': 890971913173991426,
  'retweet_count': 2158,
  'favorite_count': 12041},
 {'tweet_id': 890729181411237888,
  'retweet_count': 16716,
  'favorite_count': 56848},
 {'tweet_id': 890609185150312448,
  'retweet_count': 4429,
  'favorite_count': 28226},
 {'tweet_id': 890240255349198849,
  'retweet_count': 7711,
  'favorite_count': 32467},
 {'tweet_id': 890006608113172480,
  'retweet_count': 7624,
  'favorite_count': 31166},
 {'tweet_id': 889880896479866881,
  'retwe

In [7]:
### convert tweet_api_data from dictionary to dataframe using panda
more_data_df = pd.DataFrame(more_data, columns=['tweet_id', 'retweet_count', 'favorite_count'])
more_data_df.head()

tweet_id  retweet_count  favorite_count
0  892420643555336193           8853           39467
1  892177421306343426           6514           33819
2  891815181378084864           4328           25461
3  891689557279858688           8964           42908
4  891327558926688256           9774           41048

<a id='Assess'></a>
# **ASSESS**
Both Visual and Programmatic assessment would be done in this section

In this section I will be checking for the Quality and Tidiness of the these datasets


## **Visual assessment**

In [8]:
twitter_archive_df.sample(5) ## 

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
1380  701214700881756160                    NaN                  NaN   
1425  697943111201378304                    NaN                  NaN   
400   824775126675836928                    NaN                  NaN   
1540  689659372465688576                    NaN                  NaN   
371   828408677031882754                    NaN                  NaN   

                      timestamp  \
1380  2016-02-21 01:19:47 +0000   
1425  2016-02-12 00:39:39 +0000   
400   2017-01-27 00:24:48 +0000   
1540  2016-01-20 04:03:02 +0000   
371   2017-02-06 01:03:14 +0000   

                                                 source  \
1380  <a href="http://twitter.com/download/iphone" r...   
1425  <a href="http://twitter.com/download/iphone" r...   
400   <a href="http://twitter.com/download/iphone" r...   
1540  <a href="http://twitter.com/download/iphone" r...   
371   <a href="http://twitter.com/download/iphone" r...   

                                                   text  retweeted_status_id  \
1380  I'm not sure what's happening here, but it's p...                  NaN   
1425  This is Cassie. She goes door to door trying t...                  NaN   
400   This is Scooter. His lack of opposable thumbs ...                  NaN   
1540  This is Ricky. He's being escorted out of the ...                  NaN   
371   This is Bear. He went outside to play in the s...                  NaN   

      retweeted_status_user_id retweeted_status_timestamp  \
1380                       NaN                        NaN   
1425                       NaN                        NaN   
400                        NaN                        NaN   
1540                       NaN                        NaN   
371                        NaN                        NaN   

                                          expanded_urls  rating_numerator  \
1380  https://twitter.com/dog_rates/status/701214700...                12   
1425  https://twitter.com/dog_rates/status/697943111...                10   
400   https://twitter.com/dog_rates/status/824775126...                12   
1540  https://twitter.com/dog_rates/status/689659372...                 8   
371   https://twitter.com/dog_rates/status/828408677...                12   

      rating_denominator     name doggo floofer pupper puppo  
1380                  10     None  None    None   None  None  
1425                  10   Cassie  None    None   None  None  
400                   10  Scooter  None    None   None  None  
1540                  10    Ricky  None    None   None  None  
371                   10     Bear  None    None   None  None

In [9]:
image_prediction_df.sample(5)

tweet_id                                          jpg_url  \
65    667174963120574464  https://pbs.twimg.com/media/CUJIFoJWsAAL3Dc.jpg   
875   698195409219559425  https://pbs.twimg.com/media/CbB9BTqW8AEVc2A.jpg   
1929  858471635011153920  https://pbs.twimg.com/media/C-nnZBdXkAAB-wg.jpg   
1344  759099523532779520  https://pbs.twimg.com/media/Cojc_Q0WcAAqi_K.jpg   
424   674063288070742018  https://pbs.twimg.com/media/CVrA-rIWEAANxwQ.jpg   

      img_num                  p1   p1_conf  p1_dog  \
65          1          toy_poodle  0.266437    True   
875         1  Labrador_retriever  0.643690    True   
1929        1            Pembroke  0.987407    True   
1344        1   Shetland_sheepdog  0.129034    True   
424         1             ostrich  0.661176   False   

                                  p2   p2_conf  p2_dog              p3  \
65                         Chihuahua  0.243223    True        bluetick   
875   American_Staffordshire_terrier  0.102684    True       dalmatian   
1929                        Cardigan  0.008723    True         basenji   
1344                          kelpie  0.117508    True  Siberian_husky   
424                         bearskin  0.214879   False            swab   

       p3_conf  p3_dog  
65    0.072806    True  
875   0.050008    True  
1929  0.003424    True  
1344  0.106708    True  
424   0.064456   False

In [10]:
more_data_df.sample(5)

tweet_id  retweet_count  favorite_count
525   808838249661788160           3536           11271
1276  708853462201716736            745            1941
918   756303284449767430           1231            4359
1124  729854734790754305           1268            4396
2141  669970042633789440             65             317

## **Programmatic Assessment**

In [11]:
twitter_archive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [12]:
image_prediction_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [13]:
more_data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   tweet_id        2354 non-null   int64
 1   retweet_count   2354 non-null   int64
 2   favorite_count  2354 non-null   int64
dtypes: int64(3)
memory usage: 55.3 KB


### count of rows and columns


In [14]:
twitter_archive_df.shape

(2356, 17)

In [15]:
image_prediction_df.shape

(2075, 12)

In [16]:
more_data_df.shape

(2354, 3)

### checking for duplicate rows and null cells.

In [17]:
twitter_archive_df.duplicated().all()

False

In [18]:
image_prediction_df.duplicated().all()

False

In [19]:
more_data_df.duplicated().all()

False

In [20]:
twitter_archive_df.isna().sum()

tweet_id                         0
in_reply_to_status_id         2278
in_reply_to_user_id           2278
timestamp                        0
source                           0
text                             0
retweeted_status_id           2175
retweeted_status_user_id      2175
retweeted_status_timestamp    2175
expanded_urls                   59
rating_numerator                 0
rating_denominator               0
name                             0
doggo                            0
floofer                          0
pupper                           0
puppo                            0
dtype: int64

In [21]:
image_prediction_df.isna().any()

tweet_id    False
jpg_url     False
img_num     False
p1          False
p1_conf     False
p1_dog      False
p2          False
p2_conf     False
p2_dog      False
p3          False
p3_conf     False
p3_dog      False
dtype: bool

In [22]:
more_data_df.isna().sum()

tweet_id          0
retweet_count     0
favorite_count    0
dtype: int64

In [23]:
total_null_cell = twitter_archive_df.isnull().sum().sort_values(ascending = False)
## --- total number of rows in dataframe
total_rows_count = twitter_archive_df.isnull().count().sort_values(ascending =False) 
## --- total number of null cells in each column

percentage = (total_null_cell/total_rows_count).sort_values(ascending= False)
empty_cells = pd.concat([total_null_cell, percentage], axis=1, keys=['Total_empty_cells', '%_empty_cells'])
empty_cells.style.format({'%_empty_cells': "{:.2%}"})

### **More assessment**

###### for twitter_archive_df

In [24]:
twitter_archive_df.rating_numerator.value_counts().sort_values( ascending = False)

12      558
11      464
10      461
13      351
9       158
8       102
7        55
14       54
5        37
6        32
3        19
4        17
2         9
1         9
420       2
0         2
75        2
15        2
84        1
960       1
99        1
24        1
182       1
27        1
666       1
165       1
1776      1
204       1
17        1
50        1
45        1
60        1
44        1
143       1
121       1
20        1
26        1
144       1
80        1
88        1
Name: rating_numerator, dtype: int64

In [25]:
twitter_archive_df.rating_denominator.value_counts().sort_values( ascending = False)

10     2333
50        3
11        3
20        2
80        2
90        1
120       1
16        1
110       1
130       1
40        1
170       1
0         1
150       1
15        1
7         1
70        1
2         1
Name: rating_denominator, dtype: int64

In [26]:
### inspecting the name column.
twitter_archive_df.name

0        Phineas
1          Tilly
2         Archie
3          Darla
4       Franklin
          ...   
2351        None
2352           a
2353           a
2354           a
2355        None
Name: name, Length: 2356, dtype: object

In [27]:
### I visually inspected the name column and noticed that the bulk of the errors are in lowercase
### I will be inspecting it further programmatical

name_list = list(twitter_archive_df.name.unique()) ## create a list with all unique name in the name columns
name_error=[] ## this empty list will contain the names of lowercase which is predicted to be errorous
for n in name_list:
    if n[0].islower():
        name_error.append(n)
        print(n)        

such
a
quite
not
one
incredibly
mad
an
very
just
my
his
actually
getting
this
unacceptable
all
old
infuriating
the
by
officially
life
light
space


In [28]:
###  checking the content of source column  for any anomaly
twitter_archive_df.source.value_counts()

<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>     2221
<a href="http://vine.co" rel="nofollow">Vine - Make a Scene</a>                          91
<a href="http://twitter.com" rel="nofollow">Twitter Web Client</a>                       33
<a href="https://about.twitter.com/products/tweetdeck" rel="nofollow">TweetDeck</a>      11
Name: source, dtype: int64

In [29]:
### checking for the content of the text column

twitter_archive_df.text[1]

"This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10 https://t.co/0Xxu71qeIV"

In [30]:
twitter_archive_df.text[3]

'This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us https://t.co/tD36da7qLQ'

In [31]:
twitter_archive_df.text[60]

'This is Bella. She had her first beach experience this morning. Complete success. 12/10 would perform a sandy boop https://t.co/4VsFysDmiw'

In [32]:
twitter_archive_df.text.sample

<bound method NDFrame.sample of 0       This is Phineas. He's a mystical boy. Only eve...
1       This is Tilly. She's just checking pup on you....
2       This is Archie. He is a rare Norwegian Pouncin...
3       This is Darla. She commenced a snooze mid meal...
4       This is Franklin. He would like you to stop ca...
                              ...                        
2351    Here we have a 1949 1st generation vulpix. Enj...
2352    This is a purebred Piers Morgan. Loves to Netf...
2353    Here is a very happy pup. Big fan of well-main...
2354    This is a western brown Mitsubishi terrier. Up...
2355    Here we have a Japanese Irish Setter. Lost eye...
Name: text, Length: 2356, dtype: object>

The text column contains both text and url

### For image_prediction_df

In [33]:
image_prediction_df.query("p1_dog== False & p2_dog== False & p3_dog == False ").count()

tweet_id    324
jpg_url     324
img_num     324
p1          324
p1_conf     324
p1_dog      324
p2          324
p2_conf     324
p2_dog      324
p3          324
p3_conf     324
p3_dog      324
dtype: int64

In [34]:
### checking jpg_url duplicate
image_prediction_df.jpg_url.value_counts()

#some url are repeated twice 

https://pbs.twimg.com/media/CZhn-QAWwAASQan.jpg                                            2
https://pbs.twimg.com/media/Cq9guJ5WgAADfpF.jpg                                            2
https://pbs.twimg.com/ext_tw_video_thumb/807106774843039744/pu/img/8XZg1xW35Xp2J6JW.jpg    2
https://pbs.twimg.com/media/CU1zsMSUAAAS0qW.jpg                                            2
https://pbs.twimg.com/media/CsrjryzWgAAZY00.jpg                                            2
                                                                                          ..
https://pbs.twimg.com/media/CXrmMSpUwAAdeRj.jpg                                            1
https://pbs.twimg.com/media/CXrawAhWkAAWSxC.jpg                                            1
https://pbs.twimg.com/media/CXrIntsUsAEkv0d.jpg                                            1
https://pbs.twimg.com/media/CXqcOHCUQAAugTB.jpg                                            1
https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg                       

In [35]:
## number of duplicated url in jpg_url columns
(image_prediction_df.jpg_url.duplicated()).value_counts()

False    2009
True       66
Name: jpg_url, dtype: int64

## Assessment observation  

>  ___Quality issue___ 

 
 **Twitter_archive_df**



 
1). timestamp has +0000 all through the column; It is unneccessary.

2). timestamp dtype is in string(object) instead of a datetime format.

3). text column contain not just text but some url.

4). missing names in the name column.

5). the values that are inputted in lowercase in the name column are not fit for a name such as "a".

6). some denominating rating are less than 10 and greater than 10.

7). some numerator rating are invalid.

8). source column contain text and url.

9). dogname contains none instead of NaN.

10). the retweet columns contains over.

11). some columns such as;(in_reply_to_user_id,in_reply_to_status_id,retweeted_status_timestamp,retweeted_status_id,retweeted_status_user_id) have over 90% of missing values.


--



**image_prediction_df**


12). inconsistent naming style for the breeds (p1, p2 and p3)

13). there are redundant columns in the image_prediction_df. It is not needed for the analysis.

14). All p1_dog == False rows is not needed for this analysis.

15). there are 66 duplicate entries in the jpg_url column

--
   

**more_data_df**

15). tweet_id is an integer instead of string.



> ___Tidiness issue___

1).  All tables can be merged to a single column.

2).  The dog category should be collapsed into a single column.





<a id='Clean'></a>
# **Clean**

In [36]:
## duplicate all dataframe
df_archive_clean = twitter_archive_df.copy()
df_image_clean = image_prediction_df.copy()
df_more_clean = more_data_df.copy()

In [37]:
df_archive_clean.head(3)

tweet_id  in_reply_to_status_id  in_reply_to_user_id  \
0  892420643555336193                    NaN                  NaN   
1  892177421306343426                    NaN                  NaN   
2  891815181378084864                    NaN                  NaN   

                   timestamp  \
0  2017-08-01 16:23:56 +0000   
1  2017-08-01 00:17:27 +0000   
2  2017-07-31 00:18:03 +0000   

                                              source  \
0  <a href="http://twitter.com/download/iphone" r...   
1  <a href="http://twitter.com/download/iphone" r...   
2  <a href="http://twitter.com/download/iphone" r...   

                                                text  retweeted_status_id  \
0  This is Phineas. He's a mystical boy. Only eve...                  NaN   
1  This is Tilly. She's just checking pup on you....                  NaN   
2  This is Archie. He is a rare Norwegian Pouncin...                  NaN   

   retweeted_status_user_id retweeted_status_timestamp  \
0                       NaN                        NaN   
1                       NaN                        NaN   
2                       NaN                        NaN   

                                       expanded_urls  rating_numerator  \
0  https://twitter.com/dog_rates/status/892420643...                13   
1  https://twitter.com/dog_rates/status/892177421...                13   
2  https://twitter.com/dog_rates/status/891815181...                12   

   rating_denominator     name doggo floofer pupper puppo  
0                  10  Phineas  None    None   None  None  
1                  10    Tilly  None    None   None  None  
2                  10   Archie  None    None   None  None

In [38]:
df_image_clean.head()

tweet_id                                          jpg_url  \
0  666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1  666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2  666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3  666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4  666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   

   img_num                      p1   p1_conf  p1_dog                  p2  \
0        1  Welsh_springer_spaniel  0.465074    True              collie   
1        1                 redbone  0.506826    True  miniature_pinscher   
2        1         German_shepherd  0.596461    True            malinois   
3        1     Rhodesian_ridgeback  0.408143    True             redbone   
4        1      miniature_pinscher  0.560311    True          Rottweiler   

    p2_conf  p2_dog                   p3   p3_conf  p3_dog  
0  0.156665    True    Shetland_sheepdog  0.061428    True  
1  0.074192    True  Rhodesian_ridgeback  0.072010    True  
2  0.138584    True           bloodhound  0.116197    True  
3  0.360687    True   miniature_pinscher  0.222752    True  
4  0.243682    True             Doberman  0.154629    True

In [39]:
df_more_clean.head()

tweet_id  retweet_count  favorite_count
0  892420643555336193           8853           39467
1  892177421306343426           6514           33819
2  891815181378084864           4328           25461
3  891689557279858688           8964           42908
4  891327558926688256           9774           41048

## **Quality issue**


### Issue 

The retweet columns contains over 96% missing value.

#### Define:

There are too many missing value in the retweet's columns which includes in_reply_to_user_id,in_reply_to_status_id,retweeted_status_timestamp,retweeted_status_id,retweeted_status_user_id). I will have to remove this as it will not help our analysis.

#### Code

In [40]:
# Drop all columns with replies, retweets and others
df_archive_clean

col = ['in_reply_to_status_id', 'in_reply_to_user_id', 
       'retweeted_status_id', 'retweeted_status_user_id', 
       'retweeted_status_timestamp', 'expanded_urls']
df_archive_clean.drop(col, axis=1, inplace=True)

#### Test

In [41]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   tweet_id            2356 non-null   int64 
 1   timestamp           2356 non-null   object
 2   source              2356 non-null   object
 3   text                2356 non-null   object
 4   rating_numerator    2356 non-null   int64 
 5   rating_denominator  2356 non-null   int64 
 6   name                2356 non-null   object
 7   doggo               2356 non-null   object
 8   floofer             2356 non-null   object
 9   pupper              2356 non-null   object
 10  puppo               2356 non-null   object
dtypes: int64(3), object(8)
memory usage: 202.6+ KB


### Issue
 Timestamp has +0000 all through the column; It is unneccessary.
 
 #### Define
  I will remove the +0000 in the timestamp column.


#### Code

In [42]:
df_archive_clean.timestamp=df_archive_clean.timestamp.apply(lambda x: x[:-5])

### Test

In [43]:
df_archive_clean.timestamp.sample(3)

1600    2016-01-10 02:13:27 
1762    2015-12-20 18:29:43 
147     2017-05-13 16:33:49 
Name: timestamp, dtype: object

### Issue

Timestamp dtype is in string(object) instead of a datetime format.


#### Define:

The Timestamp datatype in Twitter_Archive_df is a string. Time column are to be standardized as "datetime" format.

#### Code

In [44]:
df_archive_clean.timestamp = pd.to_datetime(df_archive_clean.timestamp)

#### Test

In [45]:
df_archive_clean.timestamp.dtype

dtype('<M8[ns]')

### Issue
Date and time are in the same column(timestamp)
#### Define:
The date and time should be in seperate columns; I will be creating new columns for both Time and Date while I drop the timestamp column.


#### Code

In [46]:
## create date column
df_archive_clean['date'] = df_archive_clean.timestamp.dt.date

## create time column
df_archive_clean['time'] = df_archive_clean.timestamp.dt.time 

## convert from object dtype to datetime
df_archive_clean.date = pd.to_datetime(df_archive_clean.date)

## drop the timestamp column
df_archive_clean.drop(columns = ['timestamp'], inplace = True)

#### Test

In [47]:
df_archive_clean.info()

df_archive_clean.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   int64         
 1   source              2356 non-null   object        
 2   text                2356 non-null   object        
 3   rating_numerator    2356 non-null   int64         
 4   rating_denominator  2356 non-null   int64         
 5   name                2356 non-null   object        
 6   doggo               2356 non-null   object        
 7   floofer             2356 non-null   object        
 8   pupper              2356 non-null   object        
 9   puppo               2356 non-null   object        
 10  date                2356 non-null   datetime64[ns]
 11  time                2356 non-null   object        
dtypes: datetime64[ns](1), int64(3), object(8)
memory usage: 221.0+ KB


tweet_id                                             source  \
0  892420643555336193  <a href="http://twitter.com/download/iphone" r...   

                                                text  rating_numerator  \
0  This is Phineas. He's a mystical boy. Only eve...                13   

   rating_denominator     name doggo floofer pupper puppo       date      time  
0                  10  Phineas  None    None   None  None 2017-08-01  16:23:56

### Issue

Text column contain not just text but some url.

#### Define:
The text column in the Twitter_archive_df contains both the tweet text and some Url. I will be extracting the Text only.


#### Code

In [48]:
def text_only(a):
    a = a.split('https:')[0]
    return a ## create a function to split text from URL .

df_archive_clean['text'] = df_archive_clean['text'].apply(text_only) ## apply the function text_only

#### Test

In [49]:
for content in df_archive_clean.text.sample(3):
    print(content)

This is Butter. She can have whatever she wants forever. 12/10 would hug softly 
Happy Saturday here's a dog in a mailbox. 12/10 
This is Aspen. He's astronomically fluffy. I wouldn't stop petting this dog if the world was ending around me 11/10 


### Issue

Source column contain text and url.

#### Define:

The "source" column in the twitter_archive_df should contain only the "source/device" from which the tweet was made by the user. The values in this column is stored in a url format. I will be extracting the source/device from these urls  

#### Code

In [50]:
df_archive_clean.source=df_archive_clean.source.str.extract(r'\>(.*?)\<')

#### Test

In [51]:
df_archive_clean.source.value_counts()

Twitter for iPhone     2221
Vine - Make a Scene      91
Twitter Web Client       33
TweetDeck                11
Name: source, dtype: int64

#### Issue
Source column datatype is in Object

#### Define: 
The "Source" column is a fixed Categorical variable without middle ground intervals. It is either this or that. This column would be changed to a category datatype for better optimization and analysis.


#### Code

In [52]:
df_archive_clean.source = df_archive_clean['source'].astype('category')

#### Test

In [53]:
df_archive_clean["source"].dtype

CategoricalDtype(categories=['TweetDeck', 'Twitter Web Client', 'Twitter for iPhone',
                  'Vine - Make a Scene'],
, ordered=False)

In [54]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   int64         
 1   source              2356 non-null   category      
 2   text                2356 non-null   object        
 3   rating_numerator    2356 non-null   int64         
 4   rating_denominator  2356 non-null   int64         
 5   name                2356 non-null   object        
 6   doggo               2356 non-null   object        
 7   floofer             2356 non-null   object        
 8   pupper              2356 non-null   object        
 9   puppo               2356 non-null   object        
 10  date                2356 non-null   datetime64[ns]
 11  time                2356 non-null   object        
dtypes: category(1), datetime64[ns](1), int64(3), object(7)
memory usage: 205.1+ KB


### Issue
Tweet_id is an integer instead of string.

#### Define
The tweet_id is a unique identifier(primary key). It should be best preserved in a string format rather int to avoid problem.  


#### Code

In [55]:
df_archive_clean.tweet_id = df_archive_clean.tweet_id.astype(str)
df_image_clean.tweet_id = df_image_clean.tweet_id.astype(str)
df_more_clean.tweet_id = df_more_clean.tweet_id.astype(str)


#### Test

In [56]:
df_archive_clean.dtypes

tweet_id                      object
source                      category
text                          object
rating_numerator               int64
rating_denominator             int64
name                          object
doggo                         object
floofer                       object
pupper                        object
puppo                         object
date                  datetime64[ns]
time                          object
dtype: object

In [57]:
df_image_clean.dtypes

tweet_id     object
jpg_url      object
img_num       int64
p1           object
p1_conf     float64
p1_dog         bool
p2           object
p2_conf     float64
p2_dog         bool
p3           object
p3_conf     float64
p3_dog         bool
dtype: object

In [58]:
df_more_clean.dtypes

tweet_id          object
retweet_count      int64
favorite_count     int64
dtype: object

In [59]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   object        
 1   source              2356 non-null   category      
 2   text                2356 non-null   object        
 3   rating_numerator    2356 non-null   int64         
 4   rating_denominator  2356 non-null   int64         
 5   name                2356 non-null   object        
 6   doggo               2356 non-null   object        
 7   floofer             2356 non-null   object        
 8   pupper              2356 non-null   object        
 9   puppo               2356 non-null   object        
 10  date                2356 non-null   datetime64[ns]
 11  time                2356 non-null   object        
dtypes: category(1), datetime64[ns](1), int64(2), object(8)
memory usage: 205.1+ KB


### Issue
Some rows in the name column are wrongly captured.
#### Define 
The name column was extracted from the "text" column thus, some are wrongly captured ( e.g This is....). Ideally, this rows should contain a NaN value. I will be filling this rows with a NaN value

#### Code

In [60]:
## Re extract name from text column properly by taking the Word after "This is" ( most likely the  dog name) 

index_text_for_name = re.compile(r'(?:name(?:d)?)\s{1}(?:is\s)?([A-Za-z]+)') ## extract name from text column using regex
for index, row in df_archive_clean.iterrows():  
    if (row['name'].islower()):
        try:
            new_name = re.findall(index_text_for_name , row['text'])[0]
            df_archive_clean.loc[index,'name'] = df_archive_clean.loc[index,'name'].replace(row['name'], new_name)
        except IndexError:
            df_archive_clean.loc[index,'name'] = np.nan
## Replace none value with Nan
df_archive_clean['name'] = df_archive_clean['name'].replace('None', np.nan)

#### Test  

In [61]:
## number of rows that has None in the name column
df_archive_clean[(df_archive_clean.name) == 'None'].count()

tweet_id              0
source                0
text                  0
rating_numerator      0
rating_denominator    0
name                  0
doggo                 0
floofer               0
pupper                0
puppo                 0
date                  0
time                  0
dtype: int64

In [62]:
len(df_archive_clean[df_archive_clean.name.isin(name_error)])
0

0

In [63]:
for n in df_archive_clean.name:
    if n == "None":
        print(n)

In [64]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2356 non-null   object        
 1   source              2356 non-null   category      
 2   text                2356 non-null   object        
 3   rating_numerator    2356 non-null   int64         
 4   rating_denominator  2356 non-null   int64         
 5   name                1524 non-null   object        
 6   doggo               2356 non-null   object        
 7   floofer             2356 non-null   object        
 8   pupper              2356 non-null   object        
 9   puppo               2356 non-null   object        
 10  date                2356 non-null   datetime64[ns]
 11  time                2356 non-null   object        
dtypes: category(1), datetime64[ns](1), int64(2), object(8)
memory usage: 205.1+ KB


### Issue
Some numerator rating is invalid

#### Define
The numerator ratings are extracted from the text columns. The method of indexing has caused it to pick up some wrong number for rating, the numbers are typically >10. I will be manually correcting this errors.

####  Code

In [65]:
## Observe columns visually
pd.set_option("display.max_colwidth", None) ### display the full text on output
df_archive_clean.loc[(df_archive_clean.rating_numerator > 15), ['tweet_id', 'rating_numerator', 'rating_denominator', 'text']]

tweet_id  rating_numerator  rating_denominator  \
55    881633300179243008                17                  10   
188   855862651834028034               420                  10   
189   855860136149123072               666                  10   
290   838150277551247360               182                  10   
313   835246439529840640               960                   0   
340   832215909146226688                75                  10   
433   820690176645140481                84                  70   
516   810984652412424192                24                   7   
695   786709082849828864                75                  10   
763   778027034220126208                27                  10   
902   758467244762497024               165                 150   
979   749981277374128128              1776                  10   
1120  731156023742988288               204                 170   
1202  716439118184652801                50                  50   
1228  713900603437621249                99                  90   
1254  710658690886586372                80                  80   
1274  709198395643068416                45                  50   
1351  704054845121142784                60                  50   
1433  697463031882764288                44                  40   
1634  684225744407494656               143                 130   
1635  684222868335505415               121                 110   
1663  682808988178739200                20                  16   
1712  680494726643068929                26                  10   
1779  677716515794329600               144                 120   
1843  675853064436391936                88                  80   
2074  670842764863651840               420                  10   

                                                                                                                                             text  
55                                                    @roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s  
188                                                        @dhmontgomery We also gave snoop dogg a 420/10 but I think that predated your research  
189         @s8n You tried very hard to portray this good boy as not so good, but you have ultimately failed. His goodness shines through. 666/10  
290                                                                                                                            @markhoppus 182/10  
313                                            @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho  
340                      RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10   
433                                                                  The floofs have been released I repeat the floofs have been released. 84/70   
516                    Meet Sam. She smiles 24/7 &amp; secretly aspires to be a reindeer. \nKeep Sam smiling by clicking and sharing this link:\n  
695                                     This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10   
763    This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back   
902                                                                                       Why does this never happen at my front door... 165/150   
979                                                                                       This is Atticus. He's quite simply America af. 1776/10   
1120                                           Say hello to this unbelievably well behaved squad of doggos. 204/170 would try to pet all at once   
1202                                               This is Bluebert. He just saw that both #FinalFur match ups are split 50/50. Amazed af. 11/10   
1228         

In [66]:
## all rows with no actual rating rating in text column is ascribe NaN value.
df_archive_clean.loc[[188,189,290,433,516,902,979,1120,1202,1228,1254,1274,1351,1433,1634,1635,1663,1779,1843,2074], ['rating_numerator','rating_denominator']] = np.NaN

#### Test

In [67]:
## Test
df_nan = df_archive_clean.loc[[188,189,290,433,516,902,979,1120,1202,1228,1254,1274,1351,1433,1634,1635,1663,1779,1843,2074], ['rating_numerator','rating_denominator']]
df_nan 

rating_numerator  rating_denominator
188                NaN                 NaN
189                NaN                 NaN
290                NaN                 NaN
433                NaN                 NaN
516                NaN                 NaN
902                NaN                 NaN
979                NaN                 NaN
1120               NaN                 NaN
1202               NaN                 NaN
1228               NaN                 NaN
1254               NaN                 NaN
1274               NaN                 NaN
1351               NaN                 NaN
1433               NaN                 NaN
1634               NaN                 NaN
1635               NaN                 NaN
1663               NaN                 NaN
1779               NaN                 NaN
1843               NaN                 NaN
2074               NaN                 NaN

#### Code 
- Correcting the rows in the  rating_numerator column with actual rating in its text column that is misplaced.

- dropping the Null rows from the table.


In [68]:
## rows containing actual rating in the text column.
df_archive_clean.loc[(df_archive_clean.rating_numerator > 15), ['tweet_id', 'rating_numerator', 'rating_denominator', 'text']]

tweet_id  rating_numerator  rating_denominator  \
55    881633300179243008              17.0                10.0   
313   835246439529840640             960.0                 0.0   
340   832215909146226688              75.0                10.0   
695   786709082849828864              75.0                10.0   
763   778027034220126208              27.0                10.0   
1712  680494726643068929              26.0                10.0   

                                                                                                                                            text  
55                                                   @roushfenway These are good dogs but 17/10 is an emotional impulse rating. More like 13/10s  
313                                           @jonnysun @Lin_Manuel ok jomny I know you're excited but 960/00 isn't a valid rating, 13/10 is tho  
340                     RT @dog_rates: This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10   
695                                    This is Logan, the Chow who lived. He solemnly swears he's up to lots of good. H*ckin magical af 9.75/10   
763   This is Sophie. She's a Jubilant Bush Pupper. Super h*ckin rare. Appears at random just to smile at the locals. 11.27/10 would smile back   
1712                                                         Here we have uncovered an entire battalion of holiday puppers. Average of 11.26/10

In [69]:
## manually extracting the correct numerator rating from text column.
df_archive_clean.loc[55,'rating_numerator'] = 13
df_archive_clean.loc[313,'rating_numerator'] = 13
df_archive_clean.loc[340,'rating_numerator'] = 10
df_archive_clean.loc[695,'rating_numerator'] = 10
df_archive_clean.loc[763,'rating_numerator'] = 11
df_archive_clean.loc[1712,'rating_numerator'] = 11

In [70]:
df_archive_clean.loc[(df_archive_clean.rating_numerator > 15), ['tweet_id', 'rating_numerator', 'rating_denominator', 'text']]

df_archive_clean.dropna(subset=['rating_numerator'], how ='all', inplace = True)

#### Test

In [71]:
## check null value 
df_archive_clean.rating_numerator.isna().any()

False

In [72]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2336 non-null   object        
 1   source              2336 non-null   category      
 2   text                2336 non-null   object        
 3   rating_numerator    2336 non-null   float64       
 4   rating_denominator  2336 non-null   float64       
 5   name                1521 non-null   object        
 6   doggo               2336 non-null   object        
 7   floofer             2336 non-null   object        
 8   pupper              2336 non-null   object        
 9   puppo               2336 non-null   object        
 10  date                2336 non-null   datetime64[ns]
 11  time                2336 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(2), object(8)
memory usage: 221.5+ KB


In [73]:
## Test if index still exist in column.
try:
    df_nan = df_archive_clean.loc[[188,189,290,433,516,902,979,1120,1202,1228,1254,1274,1351,1433,1634,1635,1663,1779,1843,2074], ['rating_numerator','rating_denominator']]
except KeyError:
    print("index doesn't exist in column")

index doesn't exist in column


In [74]:
df_archive_clean.sample(5)

tweet_id               source  \
562   802600418706604034  Vine - Make a Scene   
2328  666396247373291520   Twitter for iPhone   
448   819006400881917954   Twitter for iPhone   
409   823333489516937216   Twitter for iPhone   
1214  715200624753819648   Twitter for iPhone   

                                                                                                                    text  \
562                                        This is Bailey. She has mastered the head tilt. 11/10 rather h*ckin adorable    
2328  Oh goodness. A super rare northeast Qdoba kangaroo mix. Massive feet. No pouch (disappointing). Seems alert. 9/10    
448                   This is Sunny. She was also a very good First Doggo. 14/10 would also be an absolute honor to pet    
409                                                                                                 @HistoryInPics 13/10   
1214                    This is Michelangelope. He's half coffee cup. Rare af. 12/10 would hug until someone stopped me    

      rating_numerator  rating_denominator            name  doggo floofer  \
562               11.0                10.0          Bailey   None    None   
2328               9.0                10.0             NaN   None    None   
448               14.0                10.0           Sunny  doggo    None   
409               13.0                10.0             NaN   None    None   
1214              12.0                10.0  Michelangelope   None    None   

     pupper puppo       date      time  
562    None  None 2016-11-26  19:50:26  
2328   None  None 2015-11-16  23:23:41  
448    None  None 2017-01-11  02:21:57  
409    None  None 2017-01-23  00:56:15  
1214   None  None 2016-03-30  15:34:51

## **Image Prediction Cleaning**


12). inconsistent naming style for the breeds (p1, p2 and p3)

13). there are redundant columns in the image_prediction_df. It is not needed for the analysis.

14). All p1_dog == False rows is not needed for this analysis.

15). there are 66 duplicate entries in the jpg_url column


In [75]:
df_image_clean.sample(5)

tweet_id                                          jpg_url  \
124   668237644992782336  https://pbs.twimg.com/media/CUYOl0kW4AAVe_p.jpg   
1104  722613351520608256  https://pbs.twimg.com/media/Cgc9AjMVIAERdUA.jpg   
927   702671118226825216  https://pbs.twimg.com/media/CcBjp2nWoAA8w-2.jpg   
841   694905863685980160  https://pbs.twimg.com/media/CaTNMUgUYAAB6vs.jpg   
1399  768596291618299904  https://pbs.twimg.com/media/CqqaPjqWIAAOyNL.jpg   

      img_num                  p1   p1_conf  p1_dog                p2  \
124         1                chow  0.809320    True           minivan   
1104        1  Labrador_retriever  0.530915    True  golden_retriever   
927         1          bloodhound  0.381227    True    Sussex_spaniel   
841         1             bow_tie  0.449268   False          fur_coat   
1399        1      Great_Pyrenees  0.729745    True  golden_retriever   

       p2_conf  p2_dog                   p3   p3_conf  p3_dog  
124   0.071311   False             Pekinese  0.037870    True  
1104  0.288230    True                 chow  0.044854    True  
927   0.212017    True              clumber  0.128622    True  
841   0.139099   False  black-footed_ferret  0.082232   False  
1399  0.237961    True   Labrador_retriever  0.020903    True

### Issue
Inconsistent naming style for the breeds (p1, p2 and p3)

#### Define
The image prediction columns for the dog breed prediction is inconsistent; I will be unifying it by replacing the spaces with underscore and also changing all uppercase strings to lowercase.


#### Code

In [76]:
df_image_clean['p1'].unique()

array(['Welsh_springer_spaniel', 'redbone', 'German_shepherd',
       'Rhodesian_ridgeback', 'miniature_pinscher',
       'Bernese_mountain_dog', 'box_turtle', 'chow', 'shopping_cart',
       'miniature_poodle', 'golden_retriever', 'Gordon_setter',
       'Walker_hound', 'pug', 'bloodhound', 'Lhasa', 'English_setter',
       'hen', 'desktop_computer', 'Italian_greyhound', 'Maltese_dog',
       'three-toed_sloth', 'ox', 'malamute', 'guinea_pig',
       'soft-coated_wheaten_terrier', 'Chihuahua',
       'black-and-tan_coonhound', 'coho', 'toy_terrier',
       'Blenheim_spaniel', 'Pembroke', 'llama',
       'Chesapeake_Bay_retriever', 'curly-coated_retriever', 'dalmatian',
       'Ibizan_hound', 'Border_collie', 'Labrador_retriever', 'seat_belt',
       'snail', 'miniature_schnauzer', 'Airedale', 'triceratops', 'swab',
       'hay', 'hyena', 'jigsaw_puzzle', 'West_Highland_white_terrier',
       'toy_poodle', 'giant_schnauzer', 'vizsla', 'vacuum', 'Rottweiler',
       'Siberian_husky', 't

In [77]:
df_image_clean['p2'].unique()

array(['collie', 'miniature_pinscher', 'malinois', 'redbone',
       'Rottweiler', 'English_springer', 'mud_turtle', 'Tibetan_mastiff',
       'shopping_basket', 'komondor', 'Yorkshire_terrier',
       'English_foxhound', 'bull_mastiff', 'German_shepherd', 'Shih-Tzu',
       'Newfoundland', 'cock', 'desk', 'toy_terrier', 'toy_poodle',
       'otter', 'Chesapeake_Bay_retriever', 'Siberian_husky', 'skunk',
       'Afghan_hound', 'bloodhound', 'barracouta', 'papillon',
       'cocker_spaniel', 'chow', 'Irish_terrier', 'chain_saw', 'beagle',
       'giant_schnauzer', 'Labrador_retriever', 'Pembroke', 'Chihuahua',
       'Weimaraner', 'slug', 'Brittany_spaniel', 'standard_schnauzer',
       'teddy', 'armadillo', 'African_hunting_dog', 'vizsla', 'doormat',
       'pug', 'Italian_greyhound', 'Samoyed', 'Pomeranian',
       'miniature_poodle', 'Lakeland_terrier', 'Irish_setter', 'swab',
       'malamute', 'bath_towel', 'Border_collie', 'Leonberg', 'drake',
       'French_bulldog', 'ice_bear', 

In [78]:
df_image_clean['p3'].unique()

array(['Shetland_sheepdog', 'Rhodesian_ridgeback', 'bloodhound',
       'miniature_pinscher', 'Doberman', 'Greater_Swiss_Mountain_dog',
       'terrapin', 'fur_coat', 'golden_retriever',
       'soft-coated_wheaten_terrier', 'Labrador_retriever', 'Pekinese',
       'Ibizan_hound', 'French_bulldog', 'malinois', 'Dandie_Dinmont',
       'borzoi', 'partridge', 'bookcase', 'basenji', 'miniature_poodle',
       'great_grey_owl', 'groenendael', 'Eskimo_dog', 'hamster', 'briard',
       'papillon', 'flat-coated_retriever', 'gar', 'Chihuahua',
       'Shih-Tzu', 'Pomeranian', 'dingo', 'power_drill', 'Saluki',
       'Great_Pyrenees', 'West_Highland_white_terrier', 'collie',
       'toy_poodle', 'vizsla', 'acorn', 'giant_schnauzer', 'teddy',
       'common_iguana', 'wig', 'water_buffalo', 'coyote', 'seat_belt',
       'kelpie', 'space_heater', 'Brabancon_griffon', 'standard_poodle',
       'beagle', 'Irish_water_spaniel', 'bluetick', 'Weimaraner',
       'Chesapeake_Bay_retriever', 'toilet_tiss

In [79]:
## using lambda to replace underscore.

df_image_clean.p1 = list(map(lambda x: x.replace(' ', '_'),
                                 df_image_clean.p1))
df_image_clean.p2 = list(map(lambda x: x.replace(' ', '_'),
                                 df_image_clean.p2))
df_image_clean.p3 = list(map(lambda x: x.replace(' ', '_'),
                                 df_image_clean.p3))


### Replace all uppercase with lowercase character
df_image_clean.p1 = df_image_clean.p1.str.lower()
df_image_clean.p2 = df_image_clean.p2.str.lower()
df_image_clean.p3 = df_image_clean.p3.str.lower()

#### Test

In [80]:
df_image_clean['p1'].unique()

array(['welsh_springer_spaniel', 'redbone', 'german_shepherd',
       'rhodesian_ridgeback', 'miniature_pinscher',
       'bernese_mountain_dog', 'box_turtle', 'chow', 'shopping_cart',
       'miniature_poodle', 'golden_retriever', 'gordon_setter',
       'walker_hound', 'pug', 'bloodhound', 'lhasa', 'english_setter',
       'hen', 'desktop_computer', 'italian_greyhound', 'maltese_dog',
       'three-toed_sloth', 'ox', 'malamute', 'guinea_pig',
       'soft-coated_wheaten_terrier', 'chihuahua',
       'black-and-tan_coonhound', 'coho', 'toy_terrier',
       'blenheim_spaniel', 'pembroke', 'llama',
       'chesapeake_bay_retriever', 'curly-coated_retriever', 'dalmatian',
       'ibizan_hound', 'border_collie', 'labrador_retriever', 'seat_belt',
       'snail', 'miniature_schnauzer', 'airedale', 'triceratops', 'swab',
       'hay', 'hyena', 'jigsaw_puzzle', 'west_highland_white_terrier',
       'toy_poodle', 'giant_schnauzer', 'vizsla', 'vacuum', 'rottweiler',
       'siberian_husky', 't

### Issue
Non-dog images are present in the image prediction column.
#### Define
I will query all the false image prediction in the column and drop them.

#### Code


In [81]:
df_image_clean.sample(10)

tweet_id  \
568   678380236862578688   
1337  758405701903519748   
335   672205392827572224   
1186  739238157791694849   
1438  774314403806253056   
522   676582956622721024   
1066  715696743237730304   
19    666273097616637952   
1911  853639147608842240   
1644  808106460588765185   

                                                                                      jpg_url  \
568                                           https://pbs.twimg.com/media/CWoXOfSUAAA4u8g.jpg   
1337                                          https://pbs.twimg.com/media/CoZl9fXWgAMox0n.jpg   
335                                           https://pbs.twimg.com/media/CVQnPMrVAAAzShR.jpg   
1186  https://pbs.twimg.com/ext_tw_video_thumb/739238016737267712/pu/img/-tLpyiuIzD5zR1et.jpg   
1438                                          https://pbs.twimg.com/media/Cr7q1VxWIAA5Nm7.jpg   
522                                           https://pbs.twimg.com/media/CWO0m8tUwAAB901.jpg   
1066                                          https://pbs.twimg.com/media/Ce6qZC2WAAAcSoI.jpg   
19                                            https://pbs.twimg.com/media/CT8T1mtUwAA3aqm.jpg   
1911                                          https://pbs.twimg.com/media/C9i8RhhXoAAdkMT.jpg   
1644                                          https://pbs.twimg.com/media/Czb4iFRXgAIUMiN.jpg   

      img_num                         p1   p1_conf  p1_dog  \
568         1                    dogsled  0.088540   False   
1337        4   chesapeake_bay_retriever  0.702954    True   
335         1                     carton  0.952613   False   
1186        1                 eskimo_dog  0.503372    True   
1438        3                 eskimo_dog  0.596045    True   
522         1                  seat_belt  0.790028   False   
1066        1  staffordshire_bullterrier  0.427836    True   
19          1          italian_greyhound  0.176053    True   
1911        1            german_shepherd  0.509879    True   
1644        1           golden_retriever  0.426183    True   

                      p2   p2_conf  p2_dog              p3   p3_conf  p3_dog  
568           snowmobile  0.057291   False         samoyed  0.047601    True  
1337              laptop  0.092277   False        notebook  0.032727   False  
335                crate  0.035376   False             pug  0.003267    True  
1186      siberian_husky  0.390413    True        malamute  0.080901    True  
1438      siberian_husky  0.223067    True          saluki  0.036325    True  
522          boston_bull  0.196307    True  french_bulldog  0.012429    True  
1066                 pug  0.221409    True  french_bulldog  0.132135    True  
19           toy_terrier  0.111884    True         basenji  0.111152    True  
1911            malinois  0.237311    True          kelpie  0.046916    True  
1644  labrador_retriever  0.257447    True  great_pyrenees  0.126482    True

In [82]:
### not_dog for rows with all prediction is dog(False)
not_dog = df_image_clean.query("p1_dog == False & p2_dog == False & p3_dog == False").index
df_image_clean.drop(not_dog,inplace = True)

#### Test

In [83]:
len(df_image_clean.query('p1_dog == False & p2_dog == False & p3_dog == False'))

0

### Issue
Different tweet_ids have same jpg_url

#### Define 
Some Tweet_ids contains same jpg_url which could be due to retweets. Duplicated images will not be needed for this analysis, therefore, it will be dropped from the table.

In [84]:
df_image_clean['jpg_url'].duplicated().sum()

60

In [85]:
### drop duplictated URL
df_image_clean.drop_duplicates('jpg_url', inplace = True)

#### Test

In [86]:
df_image_clean['jpg_url'].duplicated().sum()

0

### Issue
The prediction models picked up some more obvious object in the picture.
#### Define
The first prediction (p1) has the highest prediction confidence (p1_conf) with records but it could pick up more obvious object which might not be dog, which render the "p1_dog" FALSE. I will create a column for the likely dog breed using one of the Predictions that are true.




#### Code

In [87]:
### Create a function for the most likely breed of dog
def breed_pred(x):
    if x[0]==True:
        return x[1]
    elif x[2]==True:
        return x[3]
    else:
        return x[4]
    
df_image_clean['breed'] = df_image_clean[['p1_dog', 'p1', 'p2_dog', 'p2', 'p3', 'p3_dog']].apply(breed_pred, axis=1)   

#### Test

In [88]:
df_image_clean.head(10)

tweet_id                                          jpg_url  \
0   666020888022790149  https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg   
1   666029285002620928  https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg   
2   666033412701032449  https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg   
3   666044226329800704  https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg   
4   666049248165822465  https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg   
5   666050758794694657  https://pbs.twimg.com/media/CT5Jof1WUAEuVxN.jpg   
7   666055525042405380  https://pbs.twimg.com/media/CT5N9tpXIAAifs1.jpg   
8   666057090499244032  https://pbs.twimg.com/media/CT5PY90WoAAQGLo.jpg   
9   666058600524156928  https://pbs.twimg.com/media/CT5Qw94XAAA_2dP.jpg   
10  666063827256086533  https://pbs.twimg.com/media/CT5Vg_wXIAAXfnj.jpg   

    img_num                      p1   p1_conf  p1_dog                  p2  \
0         1  welsh_springer_spaniel  0.465074    True              collie   
1         1                 redbone  0.506826    True  miniature_pinscher   
2         1         german_shepherd  0.596461    True            malinois   
3         1     rhodesian_ridgeback  0.408143    True             redbone   
4         1      miniature_pinscher  0.560311    True          rottweiler   
5         1    bernese_mountain_dog  0.651137    True    english_springer   
7         1                    chow  0.692517    True     tibetan_mastiff   
8         1           shopping_cart  0.962465   False     shopping_basket   
9         1        miniature_poodle  0.201493    True            komondor   
10        1        golden_retriever  0.775930    True     tibetan_mastiff   

     p2_conf  p2_dog                           p3   p3_conf  p3_dog  \
0   0.156665    True            shetland_sheepdog  0.061428    True   
1   0.074192    True          rhodesian_ridgeback  0.072010    True   
2   0.138584    True                   bloodhound  0.116197    True   
3   0.360687    True           miniature_pinscher  0.222752    True   
4   0.243682    True                     doberman  0.154629    True   
5   0.263788    True   greater_swiss_mountain_dog  0.016199    True   
7   0.058279    True                     fur_coat  0.054449   False   
8   0.014594   False             golden_retriever  0.007959    True   
9   0.192305    True  soft-coated_wheaten_terrier  0.082086    True   
10  0.093718    True           labrador_retriever  0.072427    True   

                     breed  
0   welsh_springer_spaniel  
1                  redbone  
2          german_shepherd  
3      rhodesian_ridgeback  
4       miniature_pinscher  
5     bernese_mountain_dog  
7                     chow  
8         golden_retriever  
9         miniature_poodle  
10        golden_retriever

###  Issue
Unwanted image prediction confidence.


### Define
There are three image prediction columns in the table, I will create a column " p_conf" that will take is value with dependence with the corresponding conf level from with the breed column takes it value.

#### Code

In [89]:
def get_dependent_column(row):
    if row['p1'] == row['breed']:
        return row['p1_conf']
    elif row['p2'] == row['breed']:
        return row['p2_conf']
    elif row['p3'] == row['breed']:
        return row['p3_conf']
    else:
        return None
df_image_clean['p_conf'] = df_image_clean.apply(lambda row: get_dependent_column(row),axis = 1)

#### Test

In [90]:
df_image_clean.sample(5)

tweet_id                                          jpg_url  \
1432  773308824254029826  https://pbs.twimg.com/media/CrtYRMEWIAAUkCl.jpg   
1613  801958328846974976  https://pbs.twimg.com/media/CyEg2AXUsAA1Qpf.jpg   
1649  809084759137812480  https://pbs.twimg.com/media/CzpyM41UoAE1b2w.jpg   
1879  846514051647705089  https://pbs.twimg.com/media/C79sB4xXwAEvwKY.jpg   
1827  835297930240217089  https://pbs.twimg.com/media/C5eTCOVUsAAWhvc.jpg   

      img_num                         p1   p1_conf  p1_dog  \
1432        1              shopping_cart  0.572349   False   
1613        1  staffordshire_bullterrier  0.327887    True   
1649        1                     vizsla  0.911412    True   
1879        2           golden_retriever  0.650003    True   
1827        1                 rottweiler  0.341276    True   

                                  p2   p2_conf  p2_dog                  p3  \
1432              labrador_retriever  0.151406    True     shopping_basket   
1613  american_staffordshire_terrier  0.271916    True  labrador_retriever   
1649                      bloodhound  0.017134    True  labrador_retriever   
1879                        leonberg  0.065199    True     norfolk_terrier   
1827                  border_terrier  0.336220    True       gordon_setter   

       p3_conf  p3_dog                      breed    p_conf  
1432  0.107102   False         labrador_retriever  0.151406  
1613  0.247619    True  staffordshire_bullterrier  0.327887  
1649  0.011761    True                     vizsla  0.911412  
1879  0.052955    True           golden_retriever  0.650003  
1827  0.045448    True                 rottweiler  0.341276

### Issue

There are redundant columns in the image_prediction table.

### Define

Drop all redundant column from the dataframe. 

#### Code

In [91]:
## drop redundant columns in image_clean

df_image_clean.drop(columns =['img_num', 'p1', 'p1_conf', 'p1_dog', 'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog'], inplace = True) 

#### Test

In [92]:
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1691 entries, 0 to 2073
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  1691 non-null   object 
 1   jpg_url   1691 non-null   object 
 2   breed     1691 non-null   object 
 3   p_conf    1691 non-null   float64
dtypes: float64(1), object(3)
memory usage: 66.1+ KB


## **df_more_clean**

In [93]:
df_more_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   object
 1   retweet_count   2354 non-null   int64 
 2   favorite_count  2354 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 55.3+ KB


In [94]:
df_more_clean.duplicated().any()

False

The df_more_clean dataset looks good.

In [95]:
df_archive_clean.info()
df_image_clean.info()
df_more_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2355
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2336 non-null   object        
 1   source              2336 non-null   category      
 2   text                2336 non-null   object        
 3   rating_numerator    2336 non-null   float64       
 4   rating_denominator  2336 non-null   float64       
 5   name                1521 non-null   object        
 6   doggo               2336 non-null   object        
 7   floofer             2336 non-null   object        
 8   pupper              2336 non-null   object        
 9   puppo               2336 non-null   object        
 10  date                2336 non-null   datetime64[ns]
 11  time                2336 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(2), object(8)
memory usage: 286.0+ KB
<class 'pandas.core.fra

# **Tidiness Issue**

## Issue
The dog stage columns(doggo, floofer, pupper, puppo) should be collapsed into a single column stage


## Define 
I will interate through the  four columns and join the stage name into one single column stage. For rows that are empty, it will be converted to None value and the none value will be removed. 

### Code

In [96]:
## create column 'stage' by merging the four dog stages columns, replacing the empty rows wiht nan values

df_archive_clean['stage'] = df_archive_clean[['doggo', 'floofer', 'pupper', 'puppo']].apply(lambda x: ''.join(x), axis=1)
df_archive_clean['stage'].replace('', np.nan, inplace=True)
value_counts = df_archive_clean['stage'].value_counts()

value_counts



NoneNoneNoneNone        1956
NoneNonepupperNone       245
doggoNoneNoneNone         83
NoneNoneNonepuppo         29
doggoNonepupperNone       12
NoneflooferNoneNone        9
doggoNoneNonepuppo         1
doggoflooferNoneNone       1
Name: stage, dtype: int64

In [97]:
# Drop rows with 'None' in the 'dog_stage' column
df_archive_clean = df_archive_clean[df_archive_clean['stage'] != 'None']

df_archive_clean['stage'] = df_archive_clean['stage'].map(lambda x: x.replace("None",""))

### Test

In [98]:
df_archive_clean['stage'].value_counts()

                1956
pupper           245
doggo             83
puppo             29
doggopupper       12
floofer            9
doggopuppo         1
doggofloofer       1
Name: stage, dtype: int64

In [99]:
df_archive_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2355
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   tweet_id            2336 non-null   object        
 1   source              2336 non-null   category      
 2   text                2336 non-null   object        
 3   rating_numerator    2336 non-null   float64       
 4   rating_denominator  2336 non-null   float64       
 5   name                1521 non-null   object        
 6   doggo               2336 non-null   object        
 7   floofer             2336 non-null   object        
 8   pupper              2336 non-null   object        
 9   puppo               2336 non-null   object        
 10  date                2336 non-null   datetime64[ns]
 11  time                2336 non-null   object        
 12  stage               2336 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(2), o

### Code

In [100]:
# Drop columns 'doggo', 'floofer', 'pupper', and 'puppo'
df_archive_clean.drop(columns=['doggo', 'floofer', 'pupper', 'puppo','rating_denominator'], inplace=True)


### Test

In [101]:
df_archive_clean.sample(5)

tweet_id              source  \
2146  669923323644657664  Twitter for iPhone   
1159  724046343203856385  Twitter for iPhone   
486   814530161257443328  Twitter for iPhone   
1178  719551379208073216  Twitter for iPhone   
2214  668544745690562560  Twitter for iPhone   

                                                                                                                       text  \
2146    This is a spotted Lipitor Rumpelstiltskin named Alphred. He can't wait for the Turkey. 10/10 would pet really well    
1159                    This is Sarge. Not even he knows what his tongue is doing, but it's pretty damn spectacular. 10/10    
486       This is Alf. Someone just rubbed a balloon on his head. He's only a little pupset about it. 12/10 would pet well    
1178                        This is Harnold. He accidentally opened the front facing camera. 10/10 get it together Harnold    
2214  It is an honor to rate this pup. He is a Snorklhuahua from Amarillo. A true renaissance dog. Also part Rudolph 10/10    

      rating_numerator     name       date      time stage  
2146              10.0  Alphred 2015-11-26  16:59:01        
1159              10.0    Sarge 2016-04-24  01:24:35        
486               12.0      Alf 2016-12-29  17:54:58        
1178              10.0  Harnold 2016-04-11  15:43:12        
2214              10.0      NaN 2015-11-22  21:41:02

In [102]:
df_archive_clean.info()
df_more_clean.info()
df_image_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2355
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tweet_id          2336 non-null   object        
 1   source            2336 non-null   category      
 2   text              2336 non-null   object        
 3   rating_numerator  2336 non-null   float64       
 4   name              1521 non-null   object        
 5   date              2336 non-null   datetime64[ns]
 6   time              2336 non-null   object        
 7   stage             2336 non-null   object        
dtypes: category(1), datetime64[ns](1), float64(1), object(5)
memory usage: 148.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2354 entries, 0 to 2353
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tweet_id        2354 non-null   object
 1   retweet_count   2354 non-

## Issue
The three data table should be merge together into a single column

## Define
Merge all table on the tweet_id columns as primary key.

### Code


In [103]:
## merge archive and more on tweetid 

superdata = pd.merge(df_archive_clean, df_more_clean,  on = 'tweet_id', how = 'left')

In [104]:
super_data = pd.merge(superdata,df_image_clean, on = 'tweet_id', how = 'left')

In [105]:
superdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2335
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tweet_id          2336 non-null   object        
 1   source            2336 non-null   category      
 2   text              2336 non-null   object        
 3   rating_numerator  2336 non-null   float64       
 4   name              1521 non-null   object        
 5   date              2336 non-null   datetime64[ns]
 6   time              2336 non-null   object        
 7   stage             2336 non-null   object        
 8   retweet_count     2334 non-null   float64       
 9   favorite_count    2334 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(3), object(5)
memory usage: 185.0+ KB


<a id='Store'></a>

## Store Tables as CSV

In [106]:
df_archive_clean.to_csv('wrd_archive_clean_data.csv',index = False)
df_more_clean.to_csv('wrd_more_clean_tweet_archive.csv',index = False)
df_image_clean.to_csv('wrd_image_archive_clean_data.csv', index = False)
superdata.to_csv('wrd_merge_clean_data.csv', index = False)

In [107]:
superdata.head(10)

tweet_id              source  \
0  892420643555336193  Twitter for iPhone   
1  892177421306343426  Twitter for iPhone   
2  891815181378084864  Twitter for iPhone   
3  891689557279858688  Twitter for iPhone   
4  891327558926688256  Twitter for iPhone   
5  891087950875897856  Twitter for iPhone   
6  890971913173991426  Twitter for iPhone   
7  890729181411237888  Twitter for iPhone   
8  890609185150312448  Twitter for iPhone   
9  890240255349198849  Twitter for iPhone   

                                                                                                                                          text  \
0                                                       This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10    
1  This is Tilly. She's just checking pup on you. Hopes you're doing ok. If not, she's available for pats, snugs, boops, the whole bit. 13/10    
2                   This is Archie. He is a rare Norwegian Pouncing Corgo. Lives in the tall grass. You never know when one may strike. 12/10    
3                                                             This is Darla. She commenced a snooze mid meal. 13/10 happens to the best of us    
4  This is Franklin. He would like you to stop calling him "cute." He is a very fierce shark and should be respected as such. 12/10 #BarkWeek    
5  Here we have a majestic great white breaching off South Africa's coast. Absolutely h*ckin breathtaking. 13/10 (IG: tucker_marlo) #BarkWeek    
6                      Meet Jax. He enjoys ice cream so much he gets nervous around it. 13/10 help Jax enjoy more things by clicking below\n\n   
7                      When you watch your owner call another dog a good boy but then they turn back to you and say you're a great boy. 13/10    
8                  This is Zoey. She doesn't want to be one of the scary sharks. Just wants to be a snuggly pettable boatpet. 13/10 #BarkWeek    
9       This is Cassie. She is a college pup. Studying international doggo communication and stick theory. 14/10 so elegant much sophisticate    

   rating_numerator      name       date      time  stage  retweet_count  \
0              13.0   Phineas 2017-08-01  16:23:56                8853.0   
1              13.0     Tilly 2017-08-01  00:17:27                6514.0   
2              12.0    Archie 2017-07-31  00:18:03                4328.0   
3              13.0     Darla 2017-07-30  15:58:51                8964.0   
4              12.0  Franklin 2017-07-29  16:00:24                9774.0   
5              13.0       NaN 2017-07-29  00:08:17                3261.0   
6              13.0       Jax 2017-07-28  16:27:12                2158.0   
7              13.0       NaN 2017-07-28  00:22:40               16716.0   
8              13.0      Zoey 2017-07-27  16:25:51                4429.0   
9              14.0    Cassie 2017-07-26  15:59:51  doggo         7711.0   

   favorite_count  
0         39467.0  
1         33819.0  
2         25461.0  
3         42908.0  
4         41048.0  
5         20562.0  
6         12041.0  
7         56848.0  
8         28226.0  
9         32467.0

## *All visualization and insight will be done on Tableau and will be published as dashboard*

In [108]:
superdata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2336 entries, 0 to 2335
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   tweet_id          2336 non-null   object        
 1   source            2336 non-null   category      
 2   text              2336 non-null   object        
 3   rating_numerator  2336 non-null   float64       
 4   name              1521 non-null   object        
 5   date              2336 non-null   datetime64[ns]
 6   time              2336 non-null   object        
 7   stage             2336 non-null   object        
 8   retweet_count     2334 non-null   float64       
 9   favorite_count    2334 non-null   float64       
dtypes: category(1), datetime64[ns](1), float64(3), object(5)
memory usage: 185.0+ KB


In [109]:
superdata["stage"].notnull().value_counts

<bound method IndexOpsMixin.value_counts of 0       True
1       True
2       True
3       True
4       True
        ... 
2331    True
2332    True
2333    True
2334    True
2335    True
Name: stage, Length: 2336, dtype: bool>

In [110]:
superdata["stage"].unique()

array(['', 'doggo', 'puppo', 'pupper', 'floofer', 'doggopuppo',
       'doggofloofer', 'doggopupper'], dtype=object)

In [111]:
df_archive_clean["stage"].unique()

array(['', 'doggo', 'puppo', 'pupper', 'floofer', 'doggopuppo',
       'doggofloofer', 'doggopupper'], dtype=object)